In [1]:
# Written by Mohammadhoosein Mohamamdnia E: mhmnia@connect.hku.hk
# 7-20-2023

import os
import shutil
from ipywidgets import widgets, HBox, VBox, Label
from IPython.display import display, clear_output
from PIL import Image

# define the base directory and "Bad_ifgs" directory
base_dir = "GEOCml1"
bad_ifgs_dir = "Bad_ifgs"

# create "Bad_ifgs" directory if not exists
if not os.path.exists(bad_ifgs_dir):
    os.makedirs(bad_ifgs_dir)

# get a list of all directories under the base directory
sub_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# sort the directories to ensure a consistent order
sub_dirs.sort()

# keep track of the current image index
image_index = 0

def show_image():
    # read the image
    dir_name = sub_dirs[image_index]
    img_name = os.path.basename(dir_name) + ".unw.png"
    img_path = os.path.join(dir_name, img_name)
    img = Image.open(img_path)
    
    # create widget to display image
    file = open(img_path, "rb")
    image = file.read()
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=600,
        height=800,
    )
    
    # create text widgets for image index, total images and folder name
    image_info = widgets.Text(value=f"{dir_name}\nImage {image_index+1} of {len(sub_dirs)}", disabled=True)

    # clear previous output and display new one
    clear_output(wait=True)
    display(VBox([image_info, image_widget, Label("Would you like to retain this interferogram for the remainder of the processing?"), hbox]))

def show_next_image(button):
    global image_index
    image_index += 1
    if image_index >= len(sub_dirs):
        image_index = len(sub_dirs) - 1
        print("All images are processed.")
        return
    show_image()

def on_yes_button_clicked(button):
    show_next_image(button)

def on_no_button_clicked(button):
    global image_index
    # move the directory to "Bad_ifgs"
    dir_path = sub_dirs[image_index]
    dest_path = os.path.join(bad_ifgs_dir, os.path.basename(dir_path))
    shutil.move(dir_path, dest_path)

    # remove the moved directory from the list and continue with the next image
    sub_dirs.remove(dir_path)
    show_next_image(button)

def on_back_button_clicked(button):
    global image_index
    # make sure we are not already at the first image
    if image_index > 0:
        image_index -= 1
    show_image()

def on_cancel_button_clicked(button):
    clear_output(wait=True)

# create buttons
yes_button = widgets.Button(description='Yes')
no_button = widgets.Button(description='No')
back_button = widgets.Button(description='Back')
cancel_button = widgets.Button(description='Cancel')

# connect buttons with their handlers
yes_button.on_click(on_yes_button_clicked)
no_button.on_click(on_no_button_clicked)
back_button.on_click(on_back_button_clicked)
cancel_button.on_click(on_cancel_button_clicked)

# create a horizontal box to hold the buttons
hbox = HBox([yes_button, no_button, back_button, cancel_button])

# start from the first image
show_image()
